# Metrics Package Examples

This notebook demonstrates the neural_analysis metrics package with:
- Distance metrics (Euclidean, Mahalanobis, Cosine)
- Distribution comparison (6 statistical methods)
- Outlier detection (5 different algorithms)
- Visual comparisons showing strengths and weaknesses
- Ground truth validation

In [2]:
import numpy as np
import pandas as pd
from neural_analysis.metrics import (
    euclidean_distance,
    mahalanobis_distance,
    cosine_similarity,
    compare_distributions,
    compare_distribution_groups,
    filter_outlier,
)
from neural_analysis.plotting import (
    plot_scatter_2d,
    plot_scatter_3d,
)
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(42)

## 1. Distance Metrics Comparison

### 1.1 Euclidean vs Mahalanobis Distance

Euclidean distance treats all dimensions equally, while Mahalanobis accounts for correlations.

In [14]:
# Create two distributions: one spherical, one elongated
n_points = 200

# Spherical distribution (uncorrelated)
spherical = np.random.randn(n_points, 2)

# Elongated distribution (correlated)
cov_elongated = np.array([[1.0, 0.9], [0.9, 1.0]])
elongated = np.random.multivariate_normal([0, 0], cov_elongated, n_points)

# Test points at same Euclidean distance but different Mahalanobis distances
test_point_aligned = np.array([1.5, 1.5])  # Along correlation
test_point_perpendicular = np.array([1.5, -1.5])  # Perpendicular to correlation

# Compute distances
mean_elongated = np.mean(elongated, axis=0)
cov_elongated_emp = np.cov(elongated.T)

euc_aligned = euclidean_distance(test_point_aligned, mean_elongated)
euc_perp = euclidean_distance(test_point_perpendicular, mean_elongated)
maha_aligned = mahalanobis_distance(test_point_aligned, mean_elongated, cov_elongated_emp)
maha_perp = mahalanobis_distance(test_point_perpendicular, mean_elongated, cov_elongated_emp)

print(f"Test Point Aligned [1.5, 1.5]:")
print(f"  Euclidean: {euc_aligned:.3f}")
print(f"  Mahalanobis: {maha_aligned:.3f}")
print(f"\nTest Point Perpendicular [1.5, -1.5]:")
print(f"  Euclidean: {euc_perp:.3f}")
print(f"  Mahalanobis: {maha_perp:.3f}")
print(f"\nKey Insight:")
print(f"  Euclidean distances are equal: {abs(euc_aligned - euc_perp) < 0.01}")
print(f"  Mahalanobis correctly identifies perpendicular as farther: {maha_perp > maha_aligned}")

Test Point Aligned [1.5, 1.5]:
  Euclidean: 2.363
  Mahalanobis: 1.739

Test Point Perpendicular [1.5, -1.5]:
  Euclidean: 2.126
  Mahalanobis: 6.797

Key Insight:
  Euclidean distances are equal: False
  Mahalanobis correctly identifies perpendicular as farther: True


In [4]:
# Visualize with plotting module
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Spherical Distribution", "Elongated Distribution (Correlated)"),
)

# Spherical
fig.add_trace(
    go.Scatter(
        x=spherical[:, 0], y=spherical[:, 1],
        mode='markers', marker=dict(size=4, color='blue', opacity=0.5),
        name='Spherical'
    ),
    row=1, col=1
)

# Elongated
fig.add_trace(
    go.Scatter(
        x=elongated[:, 0], y=elongated[:, 1],
        mode='markers', marker=dict(size=4, color='green', opacity=0.5),
        name='Elongated'
    ),
    row=1, col=2
)

# Test points
for col, dist_name in [(1, "Spherical"), (2, "Elongated")]:
    fig.add_trace(
        go.Scatter(
            x=[test_point_aligned[0]], y=[test_point_aligned[1]],
            mode='markers', marker=dict(size=12, color='red', symbol='x'),
            name='Aligned', showlegend=(col==2)
        ),
        row=1, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=[test_point_perpendicular[0]], y=[test_point_perpendicular[1]],
            mode='markers', marker=dict(size=12, color='orange', symbol='x'),
            name='Perpendicular', showlegend=(col==2)
        ),
        row=1, col=col
    )

fig.update_xaxes(title_text="X", range=[-4, 4])
fig.update_yaxes(title_text="Y", range=[-4, 4])
fig.update_layout(height=400, showlegend=True, title_text="Distance Metric Comparison")
fig.show()

### 1.2 Cosine Similarity for Direction

Cosine similarity measures directional similarity, invariant to magnitude.

In [5]:
# Create vectors with different magnitudes but similar directions
v1 = np.array([1, 2, 3])
v2 = np.array([2, 4, 6])  # Same direction, 2x magnitude
v3 = np.array([1, 2, -3])  # Different direction
v4 = np.array([-1, -2, -3])  # Opposite direction

print("Cosine Similarity Examples:")
print(f"v1 vs v2 (scaled version): {cosine_similarity(v1, v2):.3f} → 1.0 (identical direction)")
print(f"v1 vs v3 (partial match): {cosine_similarity(v1, v3):.3f}")
print(f"v1 vs v4 (opposite): {cosine_similarity(v1, v4):.3f} → -1.0 (opposite direction)")

print(f"\nEuclidean distances (NOT scale-invariant):")
print(f"v1 vs v2: {euclidean_distance(v1, v2):.3f}")
print(f"v1 vs v3: {euclidean_distance(v1, v3):.3f}")
print(f"v1 vs v4: {euclidean_distance(v1, v4):.3f}")

Cosine Similarity Examples:
v1 vs v2 (scaled version): 1.000 → 1.0 (identical direction)
v1 vs v3 (partial match): -0.286
v1 vs v4 (opposite): -1.000 → -1.0 (opposite direction)

Euclidean distances (NOT scale-invariant):
v1 vs v2: 3.742
v1 vs v3: 6.000
v1 vs v4: 7.483


## 2. Distribution Comparison Methods

Compare 6 statistical methods on synthetic distributions with known ground truth.

In [6]:
# Create test distributions with controlled differences
n_samples = 300

# Ground truth: 4 scenarios with known similarity
scenarios = {
    "Identical": (
        np.random.randn(n_samples, 3),
        np.random.randn(n_samples, 3),
        "Same distribution"
    ),
    "Small Shift": (
        np.random.randn(n_samples, 3),
        np.random.randn(n_samples, 3) + 0.5,
        "Slight location difference"
    ),
    "Large Shift": (
        np.random.randn(n_samples, 3),
        np.random.randn(n_samples, 3) + 2.0,
        "Major location difference"
    ),
    "Different Shape": (
        np.random.randn(n_samples, 3),
        np.random.randn(n_samples, 3) * 2.0,
        "Same center, different spread"
    ),
}

# Test all metrics
metrics = ["wasserstein", "kolmogorov-smirnov", "jensen-shannon", 
           "euclidean", "mahalanobis", "cosine"]

results = {}
for metric in metrics:
    results[metric] = {}
    for scenario_name, (p1, p2, desc) in scenarios.items():
        dist = compare_distributions(p1, p2, metric=metric)
        results[metric][scenario_name] = dist

# Display as DataFrame
df_results = pd.DataFrame(results).T
print("\nDistribution Comparison Results:")
print("=" * 80)
print(df_results.round(3))
print("\nInterpretation:")
print("- Lower = more similar (except cosine: higher = more similar)")
print("- Identical should be ~0 for distance metrics, ~1 for cosine")
print("- Large Shift should show biggest difference")


Distribution Comparison Results:
                    Identical  Small Shift  Large Shift  Different Shape
wasserstein             0.383        1.342        5.813            2.415
kolmogorov-smirnov      0.103        0.233        0.700            0.203
jensen-shannon          0.343        0.341        0.347            0.344
euclidean               0.215        0.777        3.357            0.111
mahalanobis             0.221        0.761        3.487            0.068
cosine                 -0.199        0.024        0.078            0.343

Interpretation:
- Lower = more similar (except cosine: higher = more similar)
- Identical should be ~0 for distance metrics, ~1 for cosine
- Large Shift should show biggest difference


In [7]:
# Visualize the test scenarios
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=list(scenarios.keys()),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}]]
)

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
for (row, col), (scenario_name, (p1, p2, desc)) in zip(positions, scenarios.items()):
    # Plot first two dimensions
    fig.add_trace(
        go.Scatter(
            x=p1[:, 0], y=p1[:, 1],
            mode='markers', marker=dict(size=3, color='blue', opacity=0.4),
            name='Dist 1', showlegend=(row==1 and col==1)
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=p2[:, 0], y=p2[:, 1],
            mode='markers', marker=dict(size=3, color='red', opacity=0.4),
            name='Dist 2', showlegend=(row==1 and col==1)
        ),
        row=row, col=col
    )

fig.update_xaxes(title_text="Dimension 1")
fig.update_yaxes(title_text="Dimension 2")
fig.update_layout(height=600, title_text="Ground Truth Scenarios for Distribution Comparison")
fig.show()

### 2.1 Method Sensitivity Analysis

Test how each metric responds to increasing distribution shift.

In [8]:
# Vary shift magnitude
shifts = np.linspace(0, 3, 15)
p1_base = np.random.randn(200, 3)

sensitivity_results = {metric: [] for metric in metrics}

for shift in shifts:
    p2_shifted = np.random.randn(200, 3) + shift
    for metric in metrics:
        dist = compare_distributions(p1_base, p2_shifted, metric=metric)
        sensitivity_results[metric].append(dist)

# Plot sensitivity curves
fig = go.Figure()
for metric in metrics:
    fig.add_trace(go.Scatter(
        x=shifts, y=sensitivity_results[metric],
        mode='lines+markers', name=metric
    ))

fig.update_layout(
    title="Metric Sensitivity to Distribution Shift",
    xaxis_title="Shift Magnitude (std devs)",
    yaxis_title="Distance/Similarity",
    height=500
)
fig.show()

print("\nKey Observations:")
print("- Wasserstein: Linear response to shift (good for quantifying displacement)")
print("- K-S: Saturates quickly (good for detecting any difference)")
print("- Euclidean: Simple linear response (centroid distance)")
print("- Mahalanobis: Accounts for variance (more stable)")
print("- Cosine: Relatively stable (direction-based)")


Key Observations:
- Wasserstein: Linear response to shift (good for quantifying displacement)
- K-S: Saturates quickly (good for detecting any difference)
- Euclidean: Simple linear response (centroid distance)
- Mahalanobis: Accounts for variance (more stable)
- Cosine: Relatively stable (direction-based)


## 3. Outlier Detection Comparison

Test 5 methods on synthetic data with known outliers.

In [9]:
# Create ground truth data
n_inliers = 200
n_outliers = 20
n_dims = 3

# Inliers: tight cluster
inliers = np.random.randn(n_inliers, n_dims) * 0.5

# Outliers: far from cluster
outliers = np.random.randn(n_outliers, n_dims) * 3 + np.array([5, 5, 5])

# Combine
data_with_outliers = np.vstack([inliers, outliers])
ground_truth = np.concatenate([np.ones(n_inliers), np.zeros(n_outliers)]).astype(bool)

# Test all methods
methods = ["iqr", "zscore", "isolation", "lof", "elliptic"]
contamination = n_outliers / (n_inliers + n_outliers)

outlier_results = {}
for method in methods:
    filtered, mask = filter_outlier(
        data_with_outliers,
        method=method,
        contamination=contamination,
        threshold=3.0,
        return_mask=True
    )
    
    # Compute metrics
    true_positives = np.sum(mask & ground_truth)
    false_positives = np.sum(mask & ~ground_truth)
    true_negatives = np.sum(~mask & ~ground_truth)
    false_negatives = np.sum(~mask & ground_truth)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    outlier_results[method] = {
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Detected Outliers": np.sum(~mask),
        "Kept Inliers": np.sum(mask)
    }

df_outlier = pd.DataFrame(outlier_results).T
print("\nOutlier Detection Performance:")
print("=" * 80)
print(df_outlier.round(3))
print(f"\nGround Truth: {n_inliers} inliers, {n_outliers} outliers")
print("\nBest Method: Highest F1-Score balances precision and recall")


Outlier Detection Performance:
           Precision  Recall  F1-Score  Detected Outliers  Kept Inliers
iqr            1.000   1.000     1.000               20.0         200.0
zscore         0.990   1.000     0.995               18.0         202.0
isolation      1.000   1.000     1.000               20.0         200.0
lof            0.995   0.995     0.995               20.0         200.0
elliptic       1.000   1.000     1.000               20.0         200.0

Ground Truth: 200 inliers, 20 outliers

Best Method: Highest F1-Score balances precision and recall


In [10]:
# Visualize outlier detection results
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=["Original Data"] + methods,
    specs=[[{'type': 'scatter3d'}] * 3, [{'type': 'scatter3d'}] * 3]
)

# Original with ground truth
fig.add_trace(
    go.Scatter3d(
        x=inliers[:, 0], y=inliers[:, 1], z=inliers[:, 2],
        mode='markers', marker=dict(size=3, color='blue'),
        name='True Inliers', showlegend=True
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter3d(
        x=outliers[:, 0], y=outliers[:, 1], z=outliers[:, 2],
        mode='markers', marker=dict(size=5, color='red'),
        name='True Outliers', showlegend=True
    ),
    row=1, col=1
)

# Each method's results
positions = [(1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]
for (row, col), method in zip(positions, methods):
    _, mask = filter_outlier(
        data_with_outliers, method=method,
        contamination=contamination, return_mask=True
    )
    kept = data_with_outliers[mask]
    removed = data_with_outliers[~mask]
    
    fig.add_trace(
        go.Scatter3d(
            x=kept[:, 0], y=kept[:, 1], z=kept[:, 2],
            mode='markers', marker=dict(size=3, color='green', opacity=0.6),
            name=f'{method} kept', showlegend=False
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter3d(
            x=removed[:, 0], y=removed[:, 1], z=removed[:, 2],
            mode='markers', marker=dict(size=5, color='orange', symbol='x'),
            name=f'{method} removed', showlegend=False
        ),
        row=row, col=col
    )

fig.update_layout(height=800, title_text="Outlier Detection Method Comparison (3D)")
fig.show()

## 4. Group Distribution Analysis

Compare multiple groups (e.g., neural activity across trials/conditions).

In [11]:
# Simulate neural data: 3 conditions with different activity patterns
n_trials = 40
n_neurons = 50

conditions = {
    "Baseline": np.random.randn(n_trials, n_neurons) * 0.5,
    "Stimulus_A": np.random.randn(n_trials, n_neurons) * 0.5 + np.array([1.0] * n_neurons),
    "Stimulus_B": np.random.randn(n_trials, n_neurons) * 0.8 + np.array([0.5] * n_neurons),
}

# Between-group comparison
between_results = compare_distribution_groups(
    conditions, compare_type="between", metric="wasserstein"
)

# Convert to distance matrix
condition_names = list(conditions.keys())
dist_matrix = np.zeros((len(condition_names), len(condition_names)))
for i, name in enumerate(condition_names):
    dist_matrix[i, :] = between_results[name]

print("\nBetween-Condition Distance Matrix (Wasserstein):")
print(pd.DataFrame(dist_matrix, index=condition_names, columns=condition_names).round(3))

# Within-group variability
inside_results = compare_distribution_groups(
    conditions, compare_type="inside", metric="euclidean"
)

print("\nWithin-Condition Variability:")
for i, name in enumerate(condition_names):
    print(f"{name}: mean={inside_results['mean'][i]:.3f}, std={inside_results['std'][i]:.3f}")


Between-Condition Distance Matrix (Wasserstein):
            Baseline  Stimulus_A  Stimulus_B
Baseline       0.000      49.747      25.551
Stimulus_A    49.747       0.000      27.205
Stimulus_B    25.551      27.205       0.000

Within-Condition Variability:
Baseline: mean=5.025, std=0.451
Stimulus_A: mean=4.926, std=0.469
Stimulus_B: mean=7.855, std=0.758


In [12]:
# Visualize as heatmap
fig = go.Figure(data=go.Heatmap(
    z=dist_matrix,
    x=condition_names,
    y=condition_names,
    colorscale='Viridis',
    text=dist_matrix.round(2),
    texttemplate='%{text}',
    textfont={"size": 14}
))

fig.update_layout(
    title="Condition Similarity Matrix (Lower = More Similar)",
    xaxis_title="Condition",
    yaxis_title="Condition",
    height=500
)
fig.show()

print("\nInterpretation:")
print("- Baseline vs Stimulus_A: Largest difference (highest activity change)")
print("- Stimulus_A vs Stimulus_B: Moderate difference")
print("- Diagonal: Zero (self-comparison)")


Interpretation:
- Baseline vs Stimulus_A: Largest difference (highest activity change)
- Stimulus_A vs Stimulus_B: Moderate difference
- Diagonal: Zero (self-comparison)


## 5. Real-World Example: Neural Manifold Analysis

Simulate neural population responses and analyze with metrics.

In [13]:
# Simulate neural population with 2D preferred direction manifold
n_neurons = 100
n_timepoints = 50
n_trials_per_angle = 20

angles = np.linspace(0, 2*np.pi, 8, endpoint=False)
neural_data = {}

for angle in angles:
    # Each angle has a preferred neural pattern
    preferred_pattern = np.array([np.cos(angle), np.sin(angle)])
    
    # Generate trials with noise
    trials = []
    for _ in range(n_trials_per_angle):
        # High-dimensional neural activity projects onto 2D manifold
        manifold_activity = preferred_pattern + np.random.randn(2) * 0.2
        # Random projection to high-dim space
        projection_matrix = np.random.randn(n_neurons, 2)
        neural_activity = projection_matrix @ manifold_activity + np.random.randn(n_neurons) * 0.1
        trials.append(neural_activity)
    
    neural_data[f"angle_{int(np.degrees(angle))}"] = np.array(trials)

# Compare adjacent angles (should be similar) vs opposite angles (should differ)
print("\nNeural Manifold Analysis:")
print("Comparing activity patterns across movement directions\n")

# Adjacent angles
dist_adjacent = compare_distributions(
    neural_data["angle_0"], neural_data["angle_45"], metric="mahalanobis"
)

# Opposite angles
dist_opposite = compare_distributions(
    neural_data["angle_0"], neural_data["angle_180"], metric="mahalanobis"
)

print(f"Distance between adjacent angles (0° vs 45°): {dist_adjacent:.3f}")
print(f"Distance between opposite angles (0° vs 180°): {dist_opposite:.3f}")
print(f"\nValidation: Opposite directions should be farther: {dist_opposite > dist_adjacent}")

# Full comparison matrix
angle_names = list(neural_data.keys())
angle_matrix = compare_distribution_groups(
    neural_data, compare_type="between", metric="euclidean"
)

matrix_vals = np.array([angle_matrix[name] for name in angle_names])
print("\nFull Angular Distance Matrix:")
print(pd.DataFrame(matrix_vals, index=angle_names, columns=angle_names).round(2))


Neural Manifold Analysis:
Comparing activity patterns across movement directions

Distance between adjacent angles (0° vs 45°): 2030.627
Distance between opposite angles (0° vs 180°): 1918.737

Validation: Opposite directions should be farther: False

Full Angular Distance Matrix:
           angle_0  angle_45  angle_90  angle_135  angle_180  angle_225  \
angle_0       0.00      3.04      3.20       3.28       2.89       2.61   
angle_45      3.04      0.00      2.96       3.42       2.96       2.91   
angle_90      3.20      2.96      0.00       3.55       2.99       3.03   
angle_135     3.28      3.42      3.55       0.00       3.37       3.10   
angle_180     2.89      2.96      2.99       3.37       0.00       3.08   
angle_225     2.61      2.91      3.03       3.10       3.08       0.00   
angle_270     3.10      3.24      3.12       3.63       3.27       3.02   
angle_315     3.62      3.76      3.35       3.76       3.44       3.34   

           angle_270  angle_315  
angle_0

## 6. Summary: Method Selection Guide

### Distance Metrics:
- **Euclidean**: Simple, interpretable, treats all dimensions equally
- **Mahalanobis**: Accounts for correlations, better for elongated clusters
- **Cosine**: Direction-based, scale-invariant, good for high-dim

### Distribution Comparison:
- **Wasserstein**: Best for quantifying displacement magnitude
- **Kolmogorov-Smirnov**: Fast, good for detecting any difference
- **Jensen-Shannon**: Symmetric, bounded, histogram-based
- **Euclidean**: Simple centroid distance
- **Mahalanobis**: Accounts for covariance structure
- **Cosine**: Direction similarity of mean vectors

### Outlier Detection:
- **LOF (default)**: Density-based, works well in most cases
- **Isolation Forest**: Fast, good for high-dim data
- **Elliptic Envelope**: Assumes Gaussian, good for symmetric clusters
- **IQR/Z-score**: Fast, simple, per-feature (limited for correlated data)

### Recommendations:
1. Start with **LOF** for outliers, **Wasserstein** for distributions
2. Use **Mahalanobis** when correlations matter
3. Use **Cosine** for direction-based similarity
4. Validate with ground truth when possible
5. Compare multiple methods for robustness